In [ ]:
import copy
# TODO: investigate death works well and insternal death stats
# TODO: fix die-adaptive lr, show usefull stat instead of logs
# TODO: Collect internal statistics of history of Neurons at their death point (like: values, how fast changed, etc.)
# "batch norm analog" - i.e. collector neuron - when things will go slow with reasonable dataset - likely cifar10, then see if speed improvement
# Separate livenet core and high-level api?

import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import livenet
importlib.reload(livenet)
core = livenet.core
from simple_log import LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
livenet.core.utils.set_seed()
var = 42
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



In [ ]:
var2 = 9
va = 3
mu = 4

In [ ]:
v1 = 1
v2 = 3
v3 = 7

In [ ]:
downscale = (8, 8)
test_x, test_y = core.datasets.to_plain(*core.datasets.get_cifar10_test(), downscale=downscale)
train_x, train_y = core.datasets.to_plain(*core.datasets.get_cifar10_train(), downscale=downscale)


In [ ]:
context = core.livenet.Context()
network = core.livenet.LiveNet.create_perceptron(train_x.shape[1], 21, 10, context)
batch_iterator = core.gen_utils.batch_iterator(train_x, train_y, batch_size=1000)
criterion = core.nets.criterion_n
optimizer = core.nets.create_optimizer(network)
optimizer.learning_rate = 0.01
trainer = core.net_trainer.Trainer(network, batch_iterator, criterion, optimizer, epoch_size=10, adaptive_lr=False)


In [ ]:
network.context.alpha_l1 = 0.001
# optimizer.learning_rate = 0.001
# network.context.learning_rate = 0.03
trainer.step(100)


In [ ]:
pred = network(test_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = test_y - pred_bin
len(diff[diff != 0]) / len(diff)


In [ ]:
%matplotlib
plt.ion()

def param_amount(param):
    val = len(param["params"])
    return val

def param_max(param):
    val = np.max(np.abs(param["params"]))
    return val

def param_picker1(param):
    try:
        val0 = param["params"]["S262->D1"].item()
    except KeyError:
        val0 = 0.
    return val0

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(trainer.history, param_amount)
plt.grid()
#plt.plot(values)

accum = lib.stat_utils.AccumStat()
accum.add_value(trainer.network.parameters())
accum.plot()


In [ ]:
for name, p in sorted(network.named_parameters(),key=lambda x: len(x[0]) + ord(x[0][0]) / 1000.):
    print(name, f"{p.item():.1f}")


In [ ]:
network._parameters


In [ ]:
lib.livenet.export_onnx(network, "/home/spometun/model.onnx")
